In [4]:
import os
import sys
from dotenv import load_dotenv

notebook_path = os.getcwd()

# One level up from the notebook
PROJECT_ROOT = os.path.abspath(os.path.join(notebook_path, "../"))

DATA_ROOT = os.path.join(PROJECT_ROOT, "data")

sys.path.insert(1, PROJECT_ROOT)
load_dotenv()

False

In [3]:
import pandas as pd

In [15]:
df = pd.read_csv(os.path.join(DATA_ROOT, "AAPL.O.csv"), index_col=0)
df.index = pd.to_datetime(df.index)

In [4]:
import yfinance as yf
import pandas as pd
from statsmodels.formula import api as sm
import plotly.graph_objects as go

In [2]:
# Define the tickers for the S&P 500 and the Magnificent 7
sp500_ticker = "^GSPC"
magnificent_7_tickers = ["AAPL", "MSFT", "GOOG", "AMZN", "NVDA", "TSLA", "META"]

# Download the historical data for the S&P 500 and the Magnificent 7
data = yf.download(sp500_ticker + " " + " ".join(magnificent_7_tickers), period="1y")

# Extract the adjusted close prices
adj_close = data['Adj Close']

# Calculate daily returns
daily_returns = adj_close.pct_change().dropna()

# Rename columns for clarity
daily_returns.columns = [col.replace(' ', '_') for col in daily_returns.columns]
daily_returns.rename(columns={'^GSPC': 'SP500'}, inplace=True)

# Run regressions for each Magnificent 7 stock
betas = {}
for stock in magnificent_7_tickers:
    model = sm.ols(formula=f"{stock} ~ SP500", data=daily_returns).fit()
    betas[stock] = model.params['SP500']

# Print the betas
print("Betas of Magnificent 7 stocks with respect to S&P 500:")
for stock, beta in betas.items():
    print(f"{stock}: {beta:.3f}")

[*********************100%%**********************]  8 of 8 completed

Betas of Magnificent 7 stocks with respect to S&P 500:
AAPL: 0.886
MSFT: 1.199
GOOG: 1.242
AMZN: 1.522
NVDA: 2.819
TSLA: 2.477
META: 1.473


In [6]:
daily_returns

,AAPL,AMZN,GOOG,META,MSFT,NVDA,TSLA,SP500
Date,,,,,,,,
2024-01-30,-0.019246,-0.014015,-0.011560,-0.002394,-0.002758,0.004947,0.003457,-0.000601
2024-01-31,-0.019358,-0.023899,-0.073505,-0.024796,-0.026946,-0.019865,-0.022444,-0.016106
2024-02-01,0.013341,0.026289,0.006418,0.011893,0.015594,0.024380,0.008383,0.012494
2024-02-02,-0.005405,0.078666,0.005816,0.203176,0.018426,0.049709,-0.005030,0.010684
2024-02-05,0.009847,-0.008731,0.009684,-0.032801,-0.013545,0.047945,-0.036454,-0.003186
...,...,...,...,...,...,...,...,...
2025-01-22,0.005345,0.018638,0.002004,0.011420,0.041307,0.044309,-0.021129,0.006138
2025-01-23,-0.000759,0.001745,-0.002250,0.020770,0.001143,0.001020,-0.006577,0.005313
2025-01-24,-0.003935,-0.002421,0.011624,0.017346,-0.005932,-0.031246,-0.014065,-0.002855


In [13]:
# Calculate daily returns
daily_returns = adj_close.pct_change().dropna()

# Rename columns for clarity
daily_returns.columns = [col.replace(' ', '_') for col in daily_returns.columns]
daily_returns.rename(columns={'^GSPC': 'SP500'}, inplace=True)

# Calculate rolling betas with a 60-day window
window_size = 60
# Initialize an empty dictionary to store rolling betas
rolling_betas = {}
for stock in magnificent_7_tickers:
    rolling_betas[stock] = []

# Loop through the rolling windows
for i in range(window_size, len(daily_returns)):
    window = daily_returns.iloc[i - window_size:i]
    for stock in magnificent_7_tickers:
        model = sm.ols(formula=f"{stock} ~ SP500", data=window).fit()
        rolling_betas[stock].append(model.params['SP500'])

# Convert the rolling betas to a DataFrame with appropriate index
rolling_betas_df = pd.DataFrame(rolling_betas, index=daily_returns.index[window_size:])

# Plotting the rolling betas using Plotly
fig = go.Figure()
for stock in magnificent_7_tickers:
    fig.add_trace(go.Scatter(x=rolling_betas_df.index, y=rolling_betas_df[stock], mode='lines', name=stock))

fig.update_layout(title='Rolling Betas of Magnificent 7 Stocks',
                  xaxis_title='Date',
                  yaxis_title='Beta',
                  hovermode='x unified')

fig.show()